In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cbow_s300/other/cbow_s300/1/cbow_s300.txt
/kaggle/input/newspaper-articles/treino.csv
/kaggle/input/newspaper-articles/teste.csv


In [7]:
art_train = pd.read_csv('/kaggle/input/newspaper-articles/treino.csv')
art_test = pd.read_csv('/kaggle/input/newspaper-articles/teste.csv')
art_train.shape, art_test.shape

((90000, 6), (20513, 6))

In [8]:
art_train.sample(2)

,title,text,date,category,subcategory,link
7833,Samsung luta para limitar danos causados por r...,"Reuniões acaloradas, férias sacrificadas e equ...",2016-06-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/10/1...
84572,Crítica: Longa 'O Alfaiate' se deixa ver e esq...,"Harry Pendel é um homem à altura do título ""O ...",2015-12-03,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/03...


In [9]:
art_train.iloc[70161].title

'Edu Lobo exibe seu melhor vocal em novo CD'

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# Exemplo de uso
texto = [
    'tenha bom final de semana',
    'tenha otimo dia',
    'tenha um pessimo dia hoje'
]

vec = CountVectorizer()
vec.fit(texto)
vec.vocabulary_

{'tenha': 8,
 'bom': 0,
 'final': 3,
 'de': 1,
 'semana': 7,
 'otimo': 5,
 'dia': 2,
 'um': 9,
 'pessimo': 6,
 'hoje': 4}

In [11]:
vec_tenha = vec.transform(['final'])
print(vec_tenha.toarray())

[[0 0 0 1 0 0 0 0 0 0]]


In [12]:
!unzip '/kaggle/input/cbow_s300/other/cbow_s300/1'

with open('/kaggle/input/cbow_s300/other/cbow_s300/1/cbow_s300.txt') as f:
    for linha in range(2):
        print(next(f))

unzip:  cannot find or open /kaggle/input/cbow_s300/other/cbow_s300/1, /kaggle/input/cbow_s300/other/cbow_s300/1.zip or /kaggle/input/cbow_s300/other/cbow_s300/1.ZIP.
929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -

Agora é preciso criar o modelo à partir de um arquivo existente em formato de chave e vetores.

## Vetorizando com CBOW

In [13]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('/kaggle/input/cbow_s300/other/cbow_s300/1/cbow_s300.txt')

In [14]:
model.get_vector('negativo')

array([ 2.75559e-01, -3.88760e-02,  3.17330e-01,  4.03845e-01,
        8.12160e-02, -1.99024e-01,  3.08280e-02, -9.26060e-02,
        1.56273e-01, -2.92164e-01, -8.55910e-02, -6.36540e-02,
        2.83530e-02,  7.97590e-02, -1.88860e-02,  2.43065e-01,
       -9.41020e-02, -1.71148e-01,  2.04216e-01,  8.38180e-02,
        1.78490e-02, -1.10855e-01, -2.46985e-01,  3.63610e-02,
       -5.08640e-02, -8.71460e-02,  6.94500e-02,  5.00000e-06,
       -1.40078e-01, -1.20928e-01,  3.16870e-02,  7.60000e-02,
       -8.29320e-02,  3.91658e-01,  2.90766e-01,  2.68610e-02,
       -5.82230e-02,  2.61978e-01,  1.69999e-01, -2.19928e-01,
       -1.91044e-01, -1.13604e-01,  1.12879e-01,  2.35312e-01,
       -1.64950e-01, -5.16630e-02,  1.61803e-01, -3.87716e-01,
       -3.46153e-01,  1.58396e-01,  8.10080e-02,  4.95060e-02,
       -1.93753e-01,  2.56301e-01, -8.23930e-02, -2.05902e-01,
        5.45055e-01, -1.41667e-01, -2.53238e-01,  2.75644e-01,
       -3.84861e-01,  2.13943e-01,  1.17160e-02, -4.843

## Análise de similaridade entre palavras

In [15]:
model.most_similar(positive=['brazil', 'argentina'])

[('bolívia', 0.5798777937889099),
 ('venezuela', 0.5760349035263062),
 ('colômbia', 0.5363108515739441),
 ('bolivia', 0.522443950176239),
 ('colombia', 0.5033947229385376),
 ('nigéria', 0.4890783131122589),
 ('eslovã¡quia', 0.4747268259525299),
 ('romãªnia', 0.46081918478012085),
 ('ã�ndia', 0.45461294054985046),
 ('suã©cia', 0.447197824716568)]

In [16]:
model.most_similar('negativo')

[('positivo', 0.8318287134170532),
 ('significativo', 0.5321436524391174),
 ('adverso', 0.5035176873207092),
 ('expressivo', 0.461646169424057),
 ('desastroso', 0.45677852630615234),
 ('satisfatório', 0.43465495109558105),
 ('nulo', 0.4261943995952606),
 ('desfavorável', 0.41955530643463135),
 ('fraco', 0.4193671941757202),
 ('insatisfatório', 0.41460177302360535)]

In [17]:
# nuvens -> nuvem : estrelas -> estrela
# nuvens + estrela - nuvem = estrelas

model.most_similar(positive=['nuvens', 'estrela'], negative=['nuvem'])

[('estrelas', 0.5497430562973022),
 ('plêiades', 0.3791979253292084),
 ('colinas', 0.3746805489063263),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.37341946363449097),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727400779724),
 ('cigarras', 0.36065396666526794),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

In [18]:
model.most_similar(positive=['professor', 'mulher'], negative=['homem'])

[('professora', 0.6192208528518677),
 ('aluna', 0.5449554324150085),
 ('esposa', 0.4978230893611908),
 ('ex-aluna', 0.4884248673915863),
 ('namorada', 0.4737858772277832),
 ('enfermeira', 0.4728144407272339),
 ('filha', 0.467373788356781),
 ('irmã', 0.4584590792655945),
 ('ex-namorada', 0.45824769139289856),
 ('ex-professora', 0.45104706287384033)]

## Vetorização de texto

In [19]:
art_train.title.loc[12]

"Daniel Craig será stormtrooper em novo 'Star Wars', diz ator"

tokenizar os textos retirando pontuações

In [20]:
import nltk
import string

def tokenizador(text):
    text = text.lower()
    lista_alfanumericos = []
    for token_valido in nltk.word_tokenize(text):
        if token_valido in string.punctuation: continue
        lista_alfanumericos.append(token_valido)
    
    return lista_alfanumericos

#tokenizador('Texto, exemplo')

In [21]:
def vetores_por_soma(n_palavras):
    vetor_resultante = np.zeros(300)
    for pn in n_palavras:
        try:
            vetor_resultante += model.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = '0'*len(pn)
                vetor_resultante += model.get_vector(pn)
            else:
                vetor_resultante += model.get_vector('unknown')
        
    return vetor_resultante

#print(vetores_por_soma(teste))

In [24]:
def matriz_vetores(texts):
    x = len(texts)
    y = 300
    matriz = np.zeros((x,y))
    
    for i in range(x):
        palavra_numeros = tokenizador(texts.iloc[i])
        matriz[i] = vetores_por_soma(palavra_numeros)
    
    return matriz

In [25]:
matriz_vet_train = matriz_vetores(art_train.title)
matriz_vet_test = matriz_vetores(art_test.title)
matriz_vet_train.shape, matriz_vet_test.shape

((90000, 300), (20513, 300))

# Classificando os textos com regressão logística

In [31]:
art_train.category

0            mundo
1            mundo
2            mundo
3        cotidiano
4          mercado
           ...    
89995    cotidiano
89996      esporte
89997      mercado
89998      colunas
89999      mercado
Name: category, Length: 90000, dtype: object

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 200)
lr.fit(matriz_vet_train, art_train.category)

LogisticRegression(max_iter=200)

In [34]:
lr.score(matriz_vet_test, art_test.category)

0.7964705308828548

In [41]:
art_test.category.unique()

array(['colunas', 'esporte', 'mercado', 'cotidiano', 'mundo', 'ilustrada'],
      dtype=object)

In [43]:
from sklearn.metrics import classification_report

predict = lr.predict(matriz_vet_test)
print(classification_report(art_test.category, predict))

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.61      0.79      0.69      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.13      0.88      0.23       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.74      0.86      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.68      0.82      0.70     20513
weighted avg       0.83      0.80      0.81     20513



In [45]:
from sklearn.dummy import DummyClassifier

dc = DummyClassifier()
dc.fit(matriz_vet_train, art_train.category)
predict_dc = dc.predict(matriz_vet_test)

cr_dummy = classification_report(art_test.category, predict_dc)
print(cr_dummy)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Nosso modelo está classificando melhor que um modelo qualquer.

## Testando com Skip-gram

In [47]:
model_skip = KeyedVectors.load_word2vec_format('/kaggle/input/skip_gram/other/default/1/skip_s300.txt')

In [52]:
lr_skipgram = LogisticRegression(max_iter = 250)

def vetores_por_soma_skipgram(n_palavras):
    vetor_resultante = np.zeros(300)
    for pn in n_palavras:
        try:
            vetor_resultante += model_skip.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = '0'*len(pn)
                vetor_resultante += model_skip.get_vector(pn)
            else:
                vetor_resultante += model_skip.get_vector('unknown')
        
    return vetor_resultante

def matriz_vetores_skipgram(texts):
    x = len(texts)
    y = 300
    matriz = np.zeros((x,y))
    
    for i in range(x):
        palavra_numeros = tokenizador(texts.iloc[i])
        matriz[i] = vetores_por_soma_skipgram(palavra_numeros)
    
    return matriz

matriz_vet_train_skipgram = matriz_vetores_skipgram(art_train.title)
matriz_vet_test_skipgram = matriz_vetores_skipgram(art_test.title)

#previsao
lr_skipgram.fit(matriz_vet_train_skipgram, art_train.category)
predict_skipgram = lr_skipgram.predict(matriz_vet_test_skipgram)
print(classification_report(art_test.category, predict_skipgram))

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.63      0.81      0.70      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.92      0.26       131
     mercado       0.84      0.82      0.83      5867
       mundo       0.76      0.86      0.81      2051

    accuracy                           0.81     20513
   macro avg       0.69      0.83      0.71     20513
weighted avg       0.84      0.81      0.82     20513



Skip-gram performou um pouco melhor

# Pontos importantes no projeto

* como representar palavras como um vetor;

* a criar vetores de palavras com Sklearn.

* o que é Word2Vec;

* principais diferenças entre Word2Vec e representações tradicionais;

* como transformar um arquivo .txt em um modelo Word2Vec com Gensim;

* entender o que são as representações de palavras em 300 dimensões.

* o que está por trás das 300 dimensões;

* a relacionar palavras;

* vieses nas representações Word2Vec.

* o fluxo para combinar vetores de palavras;

* a tokenizar texto removendo as pontuações;

* um método para realizar a combinação dos vetores de palavras;

* a analisar artigos para corrigir erros da vetorização de textos;

* a melhorar a função de vetorização de texto.

* como criar as matrizes de vetores dos dados de treino e teste;

* classificação de múltiplas classes com Regressão logística;

* como usar o Classification report;

* o que são Precision e Recall;

* a diferença entre a macro média e a média ponderada;

* como classificar e comparar modelos com o Dummy classifier.

* como adaptar as funções para classificar usando o Skipgram;

* a comparar os resultados entre Skipgram e CBOW;

* algumas particularidades de Skipgram e CBOW.